In [1]:
from collections import defaultdict
import torch
import numpy as np




def read_graph(file_path):
    """
    Reads a weighted directed graph from a file. Each line contains three values:
    start vertex, end vertex, and edge weight.

    Args:
        file_path (str): Path to the file containing the graph.

    Returns:
        edges (list): List of tuples representing directed edges (start, end, weight).
    """
    edges = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    start, end, weight = map(float, line.split())
                    edges.append((int(start), int(end), weight))
    except Exception as e:
        print(f"Error reading graph: {e}")
    return edges

def initialize_graph(edges):
    """Converts edge list to adjacency list and weights dictionary."""
    graph = defaultdict(list)
    weights = {}
    for u, v, w in edges:
        graph[u].append(v)
        weights[(u, v)] = w
    return graph, weights

def find_cycles_and_reduce(graph, weights, n):
    """Phase 1: Find cycles and reduce weights using a copy."""
    weights_copy = weights.copy()  # Work with a copy of weights
    removed_edges = set()
    removed_weights = {}

    while True:
        cycle = find_cycle(graph,  n)
        if not cycle:  # No cycle found
            break

        # Ensure all edges in the cycle exist in the weights dictionary
        cycle = [(u, v) for u, v in cycle if (u, v) in weights_copy]

        if not cycle:  # If no valid cycle exists, continue
            continue

        min_weight = min(weights_copy[(u, v)] for u, v in cycle)

        for u, v in cycle:
            weights_copy[(u, v)] -= min_weight
            if weights_copy[(u, v)] <= 0:
                # Ensure the edge is in the graph before removing
                if v in graph[u]:
                    graph[u].remove(v)
                    removed_edges.add((u, v))
                    removed_weights[(u, v)] = weights[(u, v)]

    return removed_edges, removed_weights

   





from collections import deque

def find_cycle(graph, n):
    """Detect a cycle in the graph using DFS and return the cycle as a list of edges."""
    visited = [False] * n
    stack = [False] * n
    parent = [-1] * n

    def dfs(v):
        visited[v] = True
        stack[v] = True
        for neighbor in graph[v]:
            if not visited[neighbor]:
                parent[neighbor] = v
                cycle = dfs(neighbor)
                if cycle:
                    return cycle
            elif stack[neighbor]:
                # Found a cycle, reconstruct it
                cycle = []
                current = v
                while current != neighbor:
                    cycle.append((parent[current], current))
                    current = parent[current]
                cycle.append((parent[neighbor], neighbor))
                return cycle
        stack[v] = False
        return None

    for i in range(n):
        if not visited[i]:
            cycle = dfs(i)
            if cycle:
                return cycle
    return None

from collections import deque

import numpy as np

def find_minimum_weight_cycle(graph, weights, n):
    """
    Find the minimum weight cycle in the graph using Floyd-Warshall.

    :param graph: Adjacency list representation of the graph.
    :param weights: Dictionary of edge weights.
    :param n: Total number of vertices in the graph.
    :return: List of edges representing the minimum weight cycle.
    """
    # Step 1: Initialize distance and predecessor matrices
    dist = np.full((n, n), float('inf'))
    pred = [[-1 for _ in range(n)] for _ in range(n)]

    # Fill in the distances based on edge weights
    for u in range(n):
        dist[u][u] = 0
        if u in graph:
            for v in graph[u]:
                dist[u][v] = weights.get((u, v), float('inf'))
                pred[u][v] = u

    # Step 2: Run Floyd-Warshall algorithm
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]

    # Step 3: Find the minimum weight cycle
    min_cycle_weight = float('inf')
    cycle = []

    for u in range(n):
        for v in range(n):
            if u != v and dist[u][v] < float('inf') and dist[v][u] < float('inf'):
                cycle_weight = dist[u][v] + dist[v][u]
                if cycle_weight < min_cycle_weight:
                    min_cycle_weight = cycle_weight
                    # Reconstruct the cycle
                    cycle = []
                    # Trace path from u to v
                    current = v
                    while current != u:
                        cycle.append((pred[u][current], current))
                        current = pred[u][current]
                    # Trace path from v back to u
                    current = u
                    while current != v:
                        cycle.append((pred[v][current], current))
                        current = pred[v][current]

    # Return the minimum weight cycle if found
    if min_cycle_weight == float('inf'):
        return None
    else:
        return cycle





def check_and_readd_edges(graph, removed_edges, n):
    """Phase 2: Check and re-add edges if they do not create a cycle."""
    
    def has_path(start, end, graph):
        """Helper function to check if there is a path from start to end using DFS."""
        visited = [False] * n
        stack = [start]
        while stack:
            node = stack.pop()
            if node == end:
                return True
            if not visited[node]:
                visited[node] = True
                stack.extend(graph[node])
        return False

    readded_edges = set()
    f = sorted(list(removed_edges), reverse=True)

    for u, v in f:
        if not has_path(v, u, graph):  # Only re-add if it doesn't create a cycle
            graph[u].append(v)
            readded_edges.add((u, v))

    return removed_edges - readded_edges

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            if self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            elif self.rank[root_x] < self.rank[root_y]:
                self.parent[root_y] = root_y
            else:
                self.parent[root_y] = root_x
                self.rank[root_x] += 1

def mwfas(file_path):
    """
    Main function to find Minimum Weighted Feedback Arc Set (MWFAS).
    :param file_path: Path to the file containing the graph.
    :return: A dictionary with metrics, updated graph, removed edges, and their weights.
    """
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _ in edges) + 1
    graph, weights = initialize_graph(edges)

    # Original graph statistics
    total_edges = len(edges)
    total_weight = sum(w for _, _, w in edges)

    # Phase 1: Reduce cycles
  #  print("Before Phase 1:")
  #  print("Graph:", graph)
  #  print("Weights:", weights)
    
    removed_edges, removed_weights = find_cycles_and_reduce(graph, weights, n)
    
   # print("After Phase 1:")
   # print("Removed edges:", removed_edges)
   # print("Removed weights:", removed_weights)

    # Phase 2: Re-add edges (if applicable)
    removed_edges = check_and_readd_edges(graph, removed_edges, n)

    # Compute final metrics
    num_removed_edges = len(removed_edges)
    total_removed_weight = sum(removed_weights.get(edge, 0) for edge in removed_edges)

#    print("Final Results:")
#    print("Removed edges:", removed_edges)
#    print("Weights of removed edges:", {edge: removed_weights.get(edge, 0) for edge in removed_edges})
#    print("Final graph:", graph)

    # Return results
    return {
        "total_edges": total_edges,
        "total_weight": total_weight,
        "num_removed_edges": num_removed_edges,
        "removed_weight": total_removed_weight,
        "final_graph": graph,
        "removed_edges": removed_edges,
        "removed_weights": {edge: removed_weights.get(edge, 0) for edge in removed_edges},
    }

    

# Example usage




In [2]:
# After feedback arc set removal
def compute_vertex_rankings(graph, weights, n):
 #   print("in the input dag to topol sort ", "graph is ",graph, "and weights are ",weights)
    """
    Compute rankings for the vertices in a DAG.
    :param graph: Adjacency list of the DAG.
    :param weights: Dictionary of edge weights.
    :param n: Total number of vertices in the graph.
    :return: A list of rankings for the vertices.
    """
    # Step 1: Calculate in-degrees
    in_degree = [0] * n
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    # Step 2: Perform topological sort using a min-heap
    from heapq import heappop, heappush
    min_heap = []
    for i in range(n):
        if in_degree[i] == 0:
            heappush(min_heap, i)

    topological_order = []
    while min_heap:
        current = heappop(min_heap)
        topological_order.append(current)
        for neighbor in graph[current]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                heappush(min_heap, neighbor)

    # Step 3: Calculate outgoing edge weight sums for all vertices
    outgoing_weights = {v: 0 for v in range(n)}
    incoming_weights = {v: 0 for v in range(n)}

    for u in graph:
        for v in graph[u]:
            outgoing_weights[u] += weights.get((u, v), 0)
            incoming_weights[v] += weights.get((u, v), 0)


    # Step 4: Assign rankings
    rankings = [-1] * n
    current_rank = 0
    for vertex in topological_order:
        rankings[vertex] = current_rank
        current_rank += 1

    # Break ties for vertices with the same ranking based on outgoing edge weights
    tied_vertices = sorted(
        [(rankings[v], -(outgoing_weights[v] - incoming_weights[v]) / 
          (outgoing_weights[v] + incoming_weights[v] if outgoing_weights[v] + incoming_weights[v] > 0 else 1), v)
         for v in range(n)],
        key=lambda x: (x[0], x[1])
    )

    scores = [0] * n
    for final_rank, (_, _, vertex) in enumerate(tied_vertices):
        scores[vertex]=n-final_rank-1

    print(scores)
    return scores

# Ensure graph is a DAG after feedback arc set removal



In [3]:
def graph_to_adjacency_matrix(graph, weights, n):
    """
    Converts a graph represented as an adjacency list with weights to an adjacency matrix.

    :param graph: Adjacency list of the graph.
    :param weights: Dictionary of edge weights where the key is a tuple (u, v) and the value is the weight.
    :param n: Number of vertices in the graph.
    :return: A torch.FloatTensor adjacency matrix with weights.
    """
    adjacency_matrix = torch.zeros((n, n))
    for u in graph:
        for v in graph[u]:
            adjacency_matrix[u, v] = weights[(u, v)] # Weighted edge from u to v
  #  print("adjacency matrix= ",adjacency_matrix)
 #   print(adjacency_matrix)
    return adjacency_matrix




In [17]:
def calculate_upset_loss(adjacency_matrix, scores, style='ratio', margin=0.01):
    """
    Calculate the upset loss for the graph rankings using adjacency matrix and scores.

    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param scores: Torch FloatTensor ranking scores (n x 1).
    :param style: Type of upset loss ('naive', 'simple', 'ratio', or 'margin').
    :param margin: Margin for margin loss (default: 0.01).
    :return: Torch FloatTensor upset loss value.
    """
    epsilon = 1e-8  # For numerical stability
    
    # Ensure scores are 2D
    if scores.ndim == 1:
        scores = scores.view(-1, 1)

    # Skew-symmetric pairwise comparison matrix (M)
    M1 = adjacency_matrix - adjacency_matrix.T

    # Normalize scores to [0, 1] range
 #   normalized_scores = (scores + 1) / 2
    normalized_scores=scores

    # Pairwise score differences (T)
    T1 = normalized_scores - normalized_scores.T

    # Edge mask: Only consider meaningful edges (where M != 0)
    edge_mask = M1 != 0

    # Compute upset loss based on the selected style
    if style == 'naive':
        # Count ranking disagreements
        upset = torch.sum(torch.sign(T1[edge_mask]) != torch.sign(M1[edge_mask])) / torch.sum(edge_mask)

    elif style == 'simple':
        # Penalize disagreement between signs
        upset = torch.mean((torch.sign(T1[edge_mask]) - torch.sign(M1[edge_mask]))**2)

    elif style == 'margin':
        # Margin-based upset loss
        upset = torch.mean(torch.nn.functional.relu(-M1[edge_mask] * (T1[edge_mask] - margin)))

    elif style == 'ratio':
        # Ratio-based differentiable upset loss
        T2 = scores + scores.T + epsilon
        T = torch.div(T1, T2)
        M2 = adjacency_matrix + adjacency_matrix.T + epsilon
        M3 = torch.div(M1, M2)  # Normalize the adjacency matrix
        powers = torch.pow((M3 - T)[edge_mask], 2)
        upset = torch.sum(powers) / torch.sum(edge_mask)
  # The average value, normalized and bounded

    else:
        raise ValueError(f"Unsupported style: {style}")

    return upset


In [18]:
def evaluate_upset_losses(file_path, rankings):
    """
    Evaluate upset losses (naive, simple, ratio, margin) for a graph and given rankings.

    :param file_path: Path to the graph file.
    :param rankings: List of rankings for the vertices.
    """
    # Step 1: Prepare Graph and Adjacency Matrix
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _ in edges) + 1
    graph, weights = initialize_graph(edges)
    adjacency_matrix = graph_to_adjacency_matrix(graph, weights, n)

    # Step 2: Convert Rankings to Scores Tensor
    scores = torch.FloatTensor(rankings).view(-1, 1)

    # Step 3: Calculate Upset Losses
    naive_loss = calculate_upset_loss(adjacency_matrix, scores, style='naive').item()
    simple_loss = calculate_upset_loss(adjacency_matrix, scores, style='simple').item()
    ratio_loss = calculate_upset_loss(adjacency_matrix, scores, style='ratio').item()
    margin_loss = calculate_upset_loss(adjacency_matrix, scores, style='margin').item()

    # Step 4: Print Results
    print("Upset Losses for the Graph Rankings:")
    print(f"Naive Upset Loss: {naive_loss:.4f}")
    print(f"Simple Upset Loss: {simple_loss:.4f}")
    print(f"Differentiable Upset Loss (Ratio): {ratio_loss:.4f}")
    print(f"Upset Margin Loss: {margin_loss:.4f}")


In [35]:
# Main Execution
import time
start_time = time.time()
# Step 1: Read the graph and initialize structures
file_path = "ComputerScience_F_All.txt"  # Replace with your dataset file
edges = read_graph(file_path)
n = max(max(u, v) for u, v, _ in edges) + 1
print("number of nodes= ",n," and number of edges= ",len(edges))
init_graph, init_weights = initialize_graph(edges)

#print("initial graph weights are", init_weights)

result = mwfas(file_path)
#print(f"Total edges in the graph: {result['total_edges']}")
#print(f"Sum of weights of the edges: {result['total_weight']:.2f}")
#print(f"Number of deleted edges: {result['num_removed_edges']}")
#print(f"Sum of weights of the deleted edges: {result['removed_weight']:.2f}")

# Step 2: Ensure the graph is a DAG by removing cycles
#removed_edges, removed_weights = find_cycles_and_reduce(graph, weights, n)
#removed_edges = check_and_readd_edges(graph, removed_edges, n)

# Remove `removed_edges` from the graph
new_graph=result['final_graph']
new_weights = {key: value for key, value in init_weights.items() if key not in result['removed_weights']}

#print("weights after removing some edges=",new_weights)
#print("graph after removing some edges=",new_graph)
# Step 3: Compute rankings for the vertices using the modified graph
final_rankings = compute_vertex_rankings(new_graph, new_weights, n)

# Step 4: Print the rankings
#print("Vertex Rankings:")
#for vertex, rank in enumerate(final_rankings):
#    print(f"Vertex {vertex}: Rank {rank}")

# Step 5: Evaluate upset losses
# Convert rankings to scores (Torch Tensor)
scores = torch.FloatTensor(final_rankings).view(-1, 1)

# Convert the graph to an adjacency matrix
adjacency_matrix = graph_to_adjacency_matrix(init_graph, init_weights,n)

# Calculate upset losses
naive_loss = calculate_upset_loss(adjacency_matrix, scores, style='naive').item()
simple_loss = calculate_upset_loss(adjacency_matrix, scores, style='simple').item()
ratio_loss = calculate_upset_loss(adjacency_matrix, scores, style='ratio').item()
margin_loss = calculate_upset_loss(adjacency_matrix, scores, style='margin').item()
# Print performance metrics
print("\nPerformance Metrics:")
print(f"Naive Upset Loss: {naive_loss:.4f}")
print(f"Simple Upset Loss: {simple_loss:.4f}")
print(f"Ratio Upset Loss: {ratio_loss:.4f}")
#print(f"Margin Upset Loss: {margin_loss:.4f}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
#print(f"Differentiable Upset Loss (Ratio): {ratio_loss:.4f}")



number of nodes=  206  and number of edges=  579
[203, 205, 177, 204, 153, 161, 143, 165, 202, 176, 175, 151, 201, 200, 174, 173, 160, 159, 172, 171, 142, 170, 169, 168, 162, 164, 167, 166, 158, 150, 157, 156, 155, 128, 152, 141, 147, 163, 199, 154, 198, 149, 148, 129, 146, 127, 145, 197, 144, 140, 139, 105, 138, 115, 126, 82, 122, 137, 196, 195, 123, 136, 116, 135, 134, 133, 132, 81, 194, 131, 103, 106, 104, 47, 130, 125, 124, 193, 121, 120, 46, 119, 78, 118, 102, 117, 114, 108, 113, 112, 70, 111, 89, 192, 110, 109, 101, 107, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 88, 83, 87, 45, 191, 86, 85, 84, 80, 79, 77, 76, 75, 74, 73, 72, 71, 69, 68, 67, 66, 65, 64, 44, 63, 190, 189, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 34, 49, 48, 43, 42, 41, 40, 188, 187, 39, 38, 37, 36, 35, 33, 32, 31, 30, 29, 28, 27, 186, 26, 25, 24, 23, 185, 184, 22, 21, 20, 183, 19, 18, 182, 17, 16, 181, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 180, 5, 4, 3, 179, 2, 1, 0, 178]

Performance Metrics:
Naive Up